In [33]:
import tensorflow as tf
import cv2
import numpy as np
from tkinter import *
from PIL import ImageTk, Image

In [34]:
# Load ML model
model = tf.keras.models.load_model('tune model.h5')

In [35]:
# Extract hand from image
def hand_extraction (sourceImage):
    # Constants for finding range of skin color in YCrCb
    min_YCrCb = np.array([0,133,77],np.uint8)
    max_YCrCb = np.array([255,173,127],np.uint8)
   
    #detect face
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(sourceImage, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        sourceImage = cv2.rectangle(sourceImage,(x,y),(x+w+30,y+h+50),(0,0,0),-1)
        roi_gray = sourceImage[y:y+h, x:x+w]
        roi_color = sourceImage[y:y+h, x:x+w]
        #cv2.imshow(' Output',roi_color)
        
    # Convert image to YCrCb
    imageYCrCb = cv2.cvtColor(sourceImage,cv2.COLOR_BGR2YCR_CB)

    # Find region with skin tone in YCrCb image
    skinRegion = cv2.inRange(imageYCrCb,min_YCrCb,max_YCrCb)
    
    # Do contour detection on skin region and choosing the one with max area
    contours, hierarchy = cv2.findContours(skinRegion, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contour_info=[]
    for c in contours:
        contour_info.append((
            c,
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda contour_info:contour_info[1], reverse=True)
    max_contour = contour_info[0]

    # Draw bounding rectangele for the contour 
    x,y,w,h = cv2.boundingRect(max_contour[0])
    cv2.rectangle(sourceImage, (x, y), (x + w+ 20, y + h + 20), (0, 255,0), 2)
    new_img=sourceImage[y:y+h+20,x:x+w+ 20]
    return new_img

In [36]:
# Change background to black
def background_removal(image):
    
    #== Parameters =======================================================================
    BLUR = 21
    CANNY_THRESH_1 = 60
    CANNY_THRESH_2 = 120  #100
    MASK_DILATE_ITER = 10
    MASK_ERODE_ITER = 10
    MASK_COLOR = (0.0,0.0,0.0) # In BGR format

    #== Processing =======================================================================

    # Convert to greyscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    #-- Edge detection -------------------------------------------------------------------
    edges = cv2.Canny(gray, CANNY_THRESH_1, CANNY_THRESH_2)
    edges = cv2.dilate(edges, None)
    edges = cv2.erode(edges, None)

    #-- Find contours in edges, sort by area ---------------------------------------------
    contour_info = []
    contours = cv2.findContours(edges, cv2.RETR_LIST, cv2.CHAIN_APPROX_NONE)[-2]
    
    for c in contours:
        contour_info.append((
            c,
            cv2.isContourConvex(c),
            cv2.contourArea(c),
        ))
    contour_info = sorted(contour_info, key=lambda c: c[2], reverse=True)
    # Get contour of max area
    max_contour = contour_info[0]

    #-- Create empty mask, draw filled polygon on it corresponding to largest contour ----
    # Mask is black, polygon is white
    mask = np.zeros(edges.shape)
    cv2.fillConvexPoly(mask, max_contour[0], (255))

    #-- Smooth mask, then blur it --------------------------------------------------------
    mask = cv2.dilate(mask, None, iterations=MASK_DILATE_ITER)
    mask = cv2.erode(mask, None, iterations=MASK_ERODE_ITER)
    mask = cv2.GaussianBlur(mask, (BLUR, BLUR), 0)
    mask_stack = np.dstack([mask]*3)    # Create 3-channel alpha mask

    #-- Blend masked img into MASK_COLOR background --------------------------------------
    mask_stack  = mask_stack.astype('float32') / 255.0          # Use float matrices,
    img         = image.astype('float32') / 255.0                 #  for easy blending

    masked = (mask_stack * img) + ((1-mask_stack) * MASK_COLOR) # Blend
    masked = (masked * 255).astype('uint8')                     # Convert back to 8-bit
    return masked

In [37]:
def get_result(temp):
###after model  mem b ta thah kaf seen
    global n , letter
    #get the index of max value in list
    m = temp.index(max(temp))
    #check the value of index of max
    if m==0:
        letter = 'م'      #frist index indcate م
    elif m==1:
        letter = 'ب'     #second index indcate ب
    elif m==2:
        letter = 'ت'      #third index indcate ت
    elif m==3:
        letter = 'ث'      #forth index indcate ث
    elif m==4:
        letter = 'ك'       #fifth index indcate ك
    elif m == 5:
        letter = 'س'       #sixth index indcate س
    
    #update the letter 
    var.set(letter) # name
    frame2.update_idletasks()

In [38]:
# Camera function
def cap():
    global sentence, img, letter ,word 
    
    # Initialize variables
    sentence = ""
    word = ""
    
    # Start video capture
    video = cv2.VideoCapture(0)
    while True:
        
        # Read frame
        check,frame=video.read()
       
        # Image processing
        # Extract hand from image
        crop = hand_extraction(frame)
        
        # Convert to greyscale
        crop = cv2.cvtColor(crop, cv2.COLOR_BGR2GRAY)
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        #cv2.imshow('frame',crop)
        
        # Deep learning model
        # Resize photo to 64x64 to be same as training data
        temp = cv2.resize(crop,dsize=(64,64))
        
        # Normalize photo
        temp = np.array(temp)/255
        
        # Reshape photo
        temp = temp.reshape(1,64,64,1)
        
        # Pass photo to model
        o = model(temp)

        # Get letter of highest prediction from model
        get_result( [o[0,0], o[0,1], o[0,2], o[0,3], o[0,4], o[0,5] ])
        
        
        # Instructions in camera window
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(gray,'Save letter press \'a\'',(60,370), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(gray,'Quit press \'q\'',(60,395), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(gray,'Remove the last letter press \'l\'',(60,420), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(gray,'Clear word press \'c\'',(60,445), font, 1,(255,255,255),2,cv2.LINE_AA)
        cv2.putText(gray,'Add word to sentence press \'space\'',(60,470), font, 1,(255,255,255),2,cv2.LINE_AA)
        
        # Show camera window
        cv2.imshow('Camera', gray)
        #cv2.imshow('frame',crop) 
        frame2.pack()     
        
        # Get key from user and perform actions
        key = cv2.waitKey(1)
        
        # Quit
        if key & 0xFF == ord('q'):
            var2.set(word) 
            var3.set(sentence) 
            frame2.update_idletasks()
            break
        
        # Add letter
        elif key==ord('a'):
            word=word+letter
            # Update labels with values of variables 
            var2.set(word) 
            frame2.update_idletasks()
        
        # Clear word
        elif key ==ord('c'):
            word="                         "
            var2.set(word)
            frame2.update_idletasks()
            word=""
            # Update labels with values of variables 
            var2.set(word) 
            frame2.update_idletasks()
        
        # Backspace
        elif key ==ord('l'):
            word= word[:-1]
            # Update labels with values of variables 
            var2.set(word)  
            frame2.update_idletasks()
        
        # Add word to sentence
        elif key ==ord(' '):
            sentence = sentence + word + ' '
            word="                          "
            var2.set(word)
            frame2.update_idletasks()
            word=""
            # Update labels with values of variables 
            var2.set(word) 
            var3.set(sentence) 
            frame2.update_idletasks()
            
    # Close camera window
    video.release()
    cv2.destroyAllWindows()

In [39]:
#funcion Copy sentence to clipboard
def copy_to_clipboard():
    global sentence
    #clear clipboard
    ro.clipboard_clear()
    #copy sentence toclipboard
    ro.clipboard_append(sentence)
    #update the GUI
    ro.update()

In [40]:
sentence = ""      # outputs the whole sentence
letter = ''        # outputs a single char
word = ""          # outputs a single word

ro = Tk()
ro.title('capture the images')
ro.geometry("300x300")
button = Button(ro, text="open camera" ,command=cap , bg = "gray" , width = 10  )    # cap command calls the webcam to start capturing
button.pack()

frame2=Frame(ro, width=250, height=250)     # create a frame with W 250 x H 250

# calls copy to    clipboard function which copies the output word to paste it afterwards. 

button2 = Button(frame2, text="copy the word to clipboard" ,command=copy_to_clipboard , bg = "gray" , width = 25  )
button2.pack()

l1 = Label(frame2, text="The Predicted Letter")
var = StringVar()    # create a string variable
var.set(letter)      # set it to "letter"
l2 = Label(frame2, textvariable = var)   # display var "letter" as l2 
l3 = Label(frame2, text="The word")

var2 = StringVar()
var2.set(word)        # do the same with "word" and store it as a var
l4 = Label(frame2, textvariable = var2)   # display it as l4

l5 = Label(frame2, text="The sentence")

var3 = StringVar()
var3.set(sentence)      # do the same with "sentence" and store it as a var
l6 = Label(frame2, textvariable = var3)   # display it as l6

l1.pack()
l2.pack()
l3.pack()
l4.pack()
l5.pack()
l6.pack()

ro.mainloop()